In [1]:
from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory import InterpolationMode
import time
import cv2 as cv
import numpy as np
from threading import Thread
from pathlib import Path
import requests
import pandas as pd
import winsound

In [2]:
def get_ip(location:str):
    file = Path(location).absolute()
    with file.open('r') as f:
        ip = f.readlines()
    return ip[0]

In [6]:
def record(file_name, action_time):
    winsound.Beep(1000,1000)
    sampling_frequency = 100  # in Hz
    record_duration = action_time  # in sec.
    #record
    trajectories_right = []
    trajectories_left = []
    start = time.time()
    while (time.time() - start) < record_duration:
        # We here get the present position for all of recorded joints
        current_point_right = [joint.present_position for joint in recorded_joints_right]
        current_point_left = [joint.present_position for joint in recorded_joints_left]

        # Add this point to the already recorded trajectories
        trajectories_right.append(current_point_right)
        trajectories_left.append(current_point_left)

        time.sleep(1 / sampling_frequency)

    path = f'action/{file_name}_right.txt'
    f = open(path, 'w')
    f.writelines(str(trajectories_right))
    f.close()

    path = f'action/{file_name}_left.txt'
    f = open(path, 'w')
    f.writelines(str(trajectories_left))
    f.close()

In [7]:
def get_data(location:str):
    file = Path(location).absolute()
    with file.open('r') as f:
        data = f.readlines()
    return eval(data[0])

In [ ]:
def read_reachy_action(location:str):
    f = open(location, "r",encoding="utf-8")
    data = f.readline()
    return eval(data)

In [ ]:
reachy_ip = get_ip("reachy_ip.txt")
reachy = ReachySDK(host=reachy_ip)
recorded_joints_right = [
    reachy.r_arm.r_shoulder_pitch,
    reachy.r_arm.r_shoulder_roll,
    reachy.r_arm.r_arm_yaw,
    reachy.r_arm.r_elbow_pitch,
    reachy.r_arm.r_forearm_yaw,
    reachy.r_arm.r_wrist_pitch,
    reachy.r_arm.r_wrist_roll,
    reachy.r_arm.r_gripper,
]

recorded_joints_left = [
    reachy.l_arm.l_shoulder_pitch,
    reachy.l_arm.l_shoulder_roll,
    reachy.l_arm.l_arm_yaw,
    reachy.l_arm.l_elbow_pitch,
    reachy.l_arm.l_forearm_yaw,
    reachy.l_arm.l_wrist_pitch,
    reachy.l_arm.l_wrist_roll,
    reachy.l_arm.l_gripper,
]

In [8]:
#read csv file
#df = pd.read_csv('User_use.csv')
#data = list(df.values[0])

data = get_data('User_use.txt')

In [ ]:
data

In [24]:
reachy_action = read_reachy_action("reachy_action.txt")
reachy_action.append(data)

In [25]:
path = 'reachy_action.txt'
f = open(path, 'w')
f.writelines(str(reachy_action))
f.close()

In [12]:
record(data[0],data[1])

In [11]:
reachy.turn_off_smoothly('r_arm')
reachy.turn_off_smoothly('l_arm')